#### Find OOV count of word2vec CBOW, skip gram and fasttext of common crawl datasets 
    1. without stemming 
    2. stemming whole dataset
    3. only stemming already OOV words in train dataset

 This experiment will be done only to train dataset

 #### Results 
 Vocabluary coverage

 | embedding | without stemming | stemming whole dataset | only stemming OOV words |
 | --- | --- | --- | --- |
 | w2v cbow  | 92.84%  1584| 83.29% 2784 | 96.17%  846 | 
 | w2v sgram  | 92.84%  1584 | 83.29%  2784| 96.17% 846 |
 | fasttext  FB |100% | 100%| |

 Dataset coverage

  | embedding | without stemming | stemming whole dataset | only stemming OOV words |
 | --- | --- | --- | --- |
 | w2v cbow  | 98.11% | 95.93%| 99% |
 | w2v sgram  | 98.11% | 95.93%| 99%  |
 | fasttext  FB | 100% | 100% | |

1. When use with embedding its best to not stemm dataset and do the improvemnt
2. expected improvement was  achived by only stemming OOV words with shorter suffix and longer sufiix combintion around 4% increasement in coverage
3. When handling OOV words assingg random vector form same doistribution for words with freq > 1 and for others assing mean embedding vector or 0
4. Leveraged fasttext rare word n-gram techniqe to get embeddings for non existed words in fasttetx vacab
 #### Further Experiments 
    1. Select on embedidng and evaluate performence of each different above processed dataset on DL model
    2. Evaluate on smaarasinghe et. al CNN as already done second method (stemming whole dataset)

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
#from sklearn.metrics import accuracy_score, f1_score, precision_score,roc_curve,roc_auc_score,confusion_matrix,recall_score
from sklearn.pipeline import Pipeline
import re
import gensim
pd.set_option('display.max_colwidth', 1000)
# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills
import sinhala_stemmer

In [14]:
# load datasets - no preprocessing
path = '../Datasets/processed/no_preprocessing/'
df_A = pd.read_csv(path+'df_A.csv')     

In [12]:
out = utills.removeExtraSpaces(df_A['comment'][0])
print(out)

මේ වේසිට නීතිය ක්‍රියාත්මක වෙන්නෙ කවදාද ?


In [15]:
# preprocess everything except stemming and save dataset
df_A_p = utills.preprocess(df_A,'comment')

In [10]:
df_A_p.to_csv("../Datasets/processed/no_stemming/df_A.csv",index=False)

In [16]:
# load datasets stemmed and preprocessed 
path = '../Datasets/processed/preprocessed/'
df_A_s = pd.read_csv(path+'df_A.csv')   


In [15]:
df_A_p[df_A_p['cleaned']==""]

,comment,label,df,cleaned
610,<>,0,A,
3070,"<3,",0,A,


In [17]:
df_A_p.drop(index =[610,3070],inplace=True)
df_A_s.drop(index =[610,3070],inplace=True)

In [ ]:
words_not_found = []
def word_avg_vector(model,words_list):
    if len(words_list) < 1:  # whole sentence has no words or nan
        return np.zeros(300)
    else:
        vectorized = []
        for word in words_list:
          if word in model:
            #print("word vector length",len(model[word]))
            vectorized.append(model[word])
          else:
            vectorized.append(np.zeros(300))
            words_not_found.append(word)
        #vectorized = [model[word] if word in model else np.zeros(300) for word in words_list]
        #print(vectorized)
        #print("vectorized array len ",len(vectorized),np.array(vectorized).shape)
    # doc = [word for word in doc if word in model.wv.index_to_key else np.random.rand(k)]
    #print('average embedding shape',np.mean(vectorized, axis=0).shape)
    return np.mean(vectorized, axis=0)

def get_embedding(df,model):
    # TODO add option to select TFIDF vs mean embedding
    # avg vector
    embeddings = df.apply(lambda x: word_avg_vector(model,x.split()))

    # tfidf weighted vector
    return list(embeddings)

In [5]:
import gensim.models as FastText
from gensim.test.utils import datapath
model_fasttext = FastText.fasttext.load_facebook_model(datapath("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/cc.si.300.bin"))

In [47]:
print("internal executive" in model_fasttext.wv.key_to_index)
print("internal executive" in model_fasttext.wv)

False
True


In [40]:
print(model_fasttext)

FastText(vocab=818830, vector_size=300, alpha=0.025)


In [12]:
# import gensim.models.fasttext as fasttext
# fasttext_model = fasttext.FastText.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/fasttext_uom/fasttext_300.w2v")

In [18]:
def load_emb_model(emb_model):
    from gensim.models import word2vec
    import gensim.models as FastText
    from gensim.test.utils import datapath
    if emb_model == "w2v_skipgram":
        model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/word2vec/word2vec_300.w2v")
    elif emb_model == "w2v_cbow":
        model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/CBOW-word2vec/cbow_300.w2v")
    elif emb_model == "fasttext":
        #FastText.load_fasttext_format("../../../corpus/analyzed/saved_models/wiki.si.bin")
        model = FastText.fasttext.load_facebook_model(datapath("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/cc.si.300.bin"))
        #model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/fasttext_300.w2v")
    else:
        print("Invalid argument. Need w2v_skipgram or w2v_cbow or fasttext as argument")
        model = None
    return model
        

In [19]:
def get_emb_index(model):
    from gensim.models import word2vec
    embeddings_index ={}
    for index, word in enumerate(model.wv.index_to_key):
        embeddings_index[word] = model.wv.get_vector(word)
    print('found %s word vectors' % len(embeddings_index))
    return embeddings_index

In [ ]:
# import numpy as np
# embeddings_index = {}
# for i, line in enumerate(open('/content/drive/My Drive/Research Project/Notebooks/cc.si.300.vec')):
#     values = line.split()
#     embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
# print('found %s word vectors' % len(embeddings_index))

In [20]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [21]:
def check_coverage(vocab, embeddings_index):
    import operator
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

### Without stemming

#### word2vec

In [22]:
vocab_p = build_vocab(df_A_p['cleaned'])

In [13]:
print({k: vocab_p[k] for k in list(vocab_p)[:5]})

{'මේ': 1153, 'වේසිට': 3, 'නීතිය': 53, 'ක්\u200dරියාත්මක': 39, 'වෙන්නෙ': 109}


In [14]:
print("word2vec CBOW : ")
model_cbow = load_emb_model("w2v_cbow")
#embed_cbow = get_emb_index(model_cbow)

word2vec CBOW : 
found 2137729 word vectors


In [50]:
embed_cbow = get_emb_index(model_cbow)

found 2137729 word vectors


In [15]:
oov_cbow = check_coverage(vocab_p, embed_cbow)
print("Number of unkown words in cbow = ", len(oov_cbow))
oov_cbow[:20]


Found embeddings for 92.84% of vocab
Found embeddings for  98.11% of all text
Number of unkown words in cbow =  1584


[('ගොසිප්පෝ', 29),
 ('බන්දුලයො', 23),
 ('ගොසිපෝ', 20),
 ('ගොසිප්පා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('ලොල්ලෙක්ට', 9),
 ('අංගජාතයක්', 8),
 ('නැකපතියා', 7),
 ('ලොල්ලගෙ', 6),
 ('හදනගමං', 5),
 ('වලලං', 5),
 ('අප්පුහාමියා', 5),
 ('ලක්මිණයා', 4),
 ('තෙවා', 4),
 ('ගවයෙක්ද', 4),
 ('හැදිගෑවිලම', 4),
 ('නැකපති', 4),
 ('නැකපතියො', 4),
 ('ගෙවලයන්ට', 4),
 ('ඩෙංගුදැයි', 4)]

In [27]:
print("word2vec skip gram : ")
model_skipgram = load_emb_model("w2v_skipgram")

In [28]:
embed_skipgram = get_emb_index(model_skipgram)
oov_skipgram = check_coverage(vocab_p, embed_skipgram)
print("Number of unkown words in cbow = ", len(oov_skipgram))
oov_skipgram[:20]

found 2137729 word vectors
Found embeddings for 92.84% of vocab
Found embeddings for  98.11% of all text
Number of unkown words in cbow =  1584


[('ගොසිප්පෝ', 29),
 ('බන්දුලයො', 23),
 ('ගොසිපෝ', 20),
 ('ගොසිප්පා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('ලොල්ලෙක්ට', 9),
 ('අංගජාතයක්', 8),
 ('නැකපතියා', 7),
 ('ලොල්ලගෙ', 6),
 ('හදනගමං', 5),
 ('වලලං', 5),
 ('අප්පුහාමියා', 5),
 ('ලක්මිණයා', 4),
 ('තෙවා', 4),
 ('ගවයෙක්ද', 4),
 ('හැදිගෑවිලම', 4),
 ('නැකපති', 4),
 ('නැකපතියො', 4),
 ('ගෙවලයන්ට', 4),
 ('ඩෙංගුදැයි', 4)]

#### fasttext

In [43]:
model_fasttext.wv.most_similar('ගොසිප්පෝ')

[('ගොසිප්වල', 0.8204402327537537),
 ('ගොසිප්දේශීය', 0.7919450402259827),
 ('ගොසිප්99', 0.7867631912231445),
 ('AMගොසිප්', 0.7855252623558044),
 ('හිරුගොසිප්', 0.7843061089515686),
 ('ගොසිප්ප්\u200dරධාන', 0.7780765295028687),
 ('ගොසිප', 0.7683893442153931),
 ('ගොසිප්9', 0.7672510147094727),
 ('ගොසිප්දේශපාලනයප්\u200dරධාන', 0.738722562789917),
 ('PMගොසිප්', 0.724074125289917)]

In [41]:
embed_fasttext['ගොසිප්පෝ']

KeyError: 'ගොසිප්පෝ'

In [49]:
def fasttext_OOV(model,oov_vocab,embed_index):
    count = 0
    for word, freq in oov_vocab:
        if word in model.wv:
            embed_index[word] = model.wv.get_vector(word)
            count+=1
    print(f"Added {count} words to embedding with rare words handling of fasttext")
    return embed_index



In [24]:
embed_fasttext = get_emb_index(model_fasttext)
oov_fasttext = check_coverage(vocab_p, embed_fasttext)
print("Number of unkown words in fasttext = ", len(oov_fasttext))
oov_fasttext[:20]

found 818830 word vectors
Found embeddings for 86.04% of vocab
Found embeddings for  96.35% of all text
Number of unkown words in fasttext =  3088


[('ගොසිප්පෝ', 29),
 ('බන්දුලයො', 23),
 ('ගොසිපෝ', 20),
 ('ගොසිප්පා', 16),
 ('ගොසිපා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('️', 12),
 ('ලොල්ලෙක්ට', 9),
 ('අංගජාතයක්', 8),
 ('ලොල්ලට', 8),
 ('පොන්සියො', 7),
 ('නැකපතියා', 7),
 ('ගවයෙක්ට', 7),
 ('ගවපාලනේ', 6),
 ('හැමිනෙන්නේ', 6),
 ('ලොල්ලගෙ', 6),
 ('මකබෑවිල', 5),
 ('බන්දුලයට', 5),
 ('හරකෙක්නෙ', 5),
 ('පර්සිව', 5)]

In [50]:
## after handling rare words
embed_fasttext_rare =fasttext_OOV(model_fasttext,oov_fasttext,embed_fasttext)
oov_fasttext_rare = check_coverage(vocab_p, embed_fasttext_rare)
print("Number of unkown words in fasttext = ", len(oov_fasttext_rare))
oov_fasttext_rare[:20]

Added 3088 words to embedding with rare words handling of fasttext
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text
Number of unkown words in fasttext =  0


[]

### With stemmed dataset

#### word2vec

In [51]:
vocab_s = build_vocab(df_A_s['cleaned'])

In [39]:
print({k: vocab_s[k] for k in list(vocab_s)[:5]})

{'මේ': 1179, 'වේසිට': 3, 'නීතිය': 53, 'ක්\u200dරියාත්': 39, 'වෙ': 179}


In [40]:
print("word2vec CBOW : ")
oov_cbow_s = check_coverage(vocab_s, embed_cbow)
print("Number of unkown words in cbow = ", len(oov_cbow_s))
oov_cbow_s[:20]

word2vec CBOW : 
Found embeddings for 83.29% of vocab
Found embeddings for  95.93% of all text
Number of unkown words in cbow =  2784


[('ගොසිප්ප', 51),
 ('කාලකණ්', 40),
 ('මැදමුල', 25),
 ('හික්ස', 20),
 ('දැම්මන', 18),
 ('කරනකො', 16),
 ('කෙනෙහිලික', 15),
 ('ජනාදිප', 15),
 ('මැහ', 14),
 ('ෆේස්බු', 14),
 ('පේනවන', 13),
 ('කලාන', 13),
 ('හදාග', 13),
 ('ගොම්ම', 11),
 ('නැකපති', 11),
 ('නිවන්ස', 11),
 ('පුන්නක්', 11),
 ('කරවෙ', 10),
 ('හැංග', 10),
 ('පෙට්\u200dරල', 10)]

In [41]:
print("word2vec skip gram : ")
oov_skipgram_s = check_coverage(vocab_s, embed_skipgram)
print("Number of unkown words in cbow = ", len(oov_skipgram_s))
oov_skipgram_s[:20]

word2vec skip gram : 
Found embeddings for 83.29% of vocab
Found embeddings for  95.93% of all text
Number of unkown words in cbow =  2784


[('ගොසිප්ප', 51),
 ('කාලකණ්', 40),
 ('මැදමුල', 25),
 ('හික්ස', 20),
 ('දැම්මන', 18),
 ('කරනකො', 16),
 ('කෙනෙහිලික', 15),
 ('ජනාදිප', 15),
 ('මැහ', 14),
 ('ෆේස්බු', 14),
 ('පේනවන', 13),
 ('කලාන', 13),
 ('හදාග', 13),
 ('ගොම්ම', 11),
 ('නැකපති', 11),
 ('නිවන්ස', 11),
 ('පුන්නක්', 11),
 ('කරවෙ', 10),
 ('හැංග', 10),
 ('පෙට්\u200dරල', 10)]

#### fasttext

In [53]:
embed_fasttext = get_emb_index(model_fasttext)
oov_fasttext_s = check_coverage(vocab_s, embed_fasttext)
print("Number of unkown words in fasttext = ", len(oov_fasttext_s))
oov_fasttext_s[:20]

found 818830 word vectors
Found embeddings for 74.57% of vocab
Found embeddings for  92.86% of all text
Number of unkown words in fasttext =  4237


[('දඩුව', 52),
 ('ගොසිප්ප', 51),
 ('මුස්ලි', 44),
 ('කාලකණ්', 40),
 ('කාලකන්', 38),
 ('මැරෙ', 33),
 ('තේරෙ', 26),
 ('මැදමුල', 25),
 ('දුප්ප', 24),
 ('බන්දුලය', 23),
 ('බිස්නස', 22),
 ('කවුර', 22),
 ('ඉන්නවන', 21),
 ('හිතාග', 20),
 ('හික්ස', 20),
 ('පරිස්ස', 19),
 ('වැරැද්', 19),
 ('මොකු', 18),
 ('දැම්මන', 18),
 ('ඉස්සෙ', 17)]

In [54]:
embed_fasttext_s_rare =fasttext_OOV(model_fasttext,oov_fasttext_s,embed_fasttext)
oov_fasttext_s_rare = check_coverage(vocab_s, embed_fasttext_s_rare)
print("Number of unkown words in fasttext = ", len(oov_fasttext_s_rare))
oov_fasttext_s_rare[:20]

Added 4237 words to embedding with rare words handling of fasttext
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text
Number of unkown words in fasttext =  0


[]

### Only on OOV words

In [71]:

def add_stem(embedding, vocab):
    
    import sinhala_stemmer
    word_len = 1
    stemmer = sinhala_stemmer.SinhalaStemmer()
    stem_count = [0,0]
    word_list = {}

    for word,freq in vocab:
        word_ls = stemmer.stem(word, True, word_len)[0]   
        word_ss = stemmer.stem(word, False, word_len)[0]  

        # longer suffix     
        if word not in embedding and word_ls  in embedding:
            embedding[word] = embedding[word_ls]
            stem_count[0]+=1
            word_list[word]=[word_ss,word_ls] 

        # shorter suffix
        elif word not in embedding and word_ss in embedding: 
            embedding[word] = embedding[word_ss]
            stem_count[1]+=1 
            word_list[word]=[word_ls,word_ss]  
    print(f"Added {stem_count[0]} words to embedding with longer suffix")
    print(f"Added {stem_count[1]} words to embedding with shorter suffix")
    print(len(embedding))  
    #print(word_list)  
    return embedding      


In [42]:
count =0
list_unk =[]
for word,freq in oov_cbow:
    if freq < 2:
        count+=1
        list_unk.append(word)
print(len(list_unk))
#print(list_unk)


1434


In [67]:
# word length 1
embed_cbow = get_emb_index(model_cbow)
embed_cbow_stem_1 = add_stem(embed_cbow,oov_cbow)
oov_cbow_1 = check_coverage(vocab_p, embed_cbow_stem_1)
print("Number of unkown words in cbow = ", len(oov_cbow_1))
oov_cbow_1[:20]

found 2137729 word vectors
Added 699 words to embedding with longer suffix
Added 39 words to embedding with shorter suffix
2138467
Found embeddings for 96.17% of vocab
Found embeddings for  99.00% of all text
Number of unkown words in cbow =  846


[('ගොසිප්පෝ', 29),
 ('ගොසිප්පා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('නැකපතියා', 7),
 ('හදනගමං', 5),
 ('වලලං', 5),
 ('නැකපති', 4),
 ('නැකපතියො', 4),
 ('ඩෙංගුදැයි', 4),
 ('ලොල්ලෙක්නෙ', 4),
 ('පික්ස්සු', 3),
 ('ලොල්ලනෙ', 3),
 ('නැගිටහල්ලා', 3),
 ('කුණුහර්පවලින්නෙ', 3),
 ('ප්\u200dරද්ශවාසී', 3),
 ('වහල්ලුනෙ', 3),
 ('ප්\u200dරොෙජට්', 3),
 ('කිරීඩකයෝ', 2),
 ('අභයාරාමයෙදි', 2),
 ('ද්ව්යන්ගනවක්', 2)]

In [69]:
# word length 5
embed_cbow = get_emb_index(model_cbow)
embed_cbow_stem_5 = add_stem(embed_cbow,oov_cbow)
oov_cbow_5 = check_coverage(vocab_p, embed_cbow_stem_5)
print("Number of unkown words in cbow = ", len(oov_cbow_5))
oov_cbow_5[:20]

found 2137729 word vectors
Added 598 words to embedding with longer suffix
Added 38 words to embedding with shorter suffix
2138365
Found embeddings for 95.71% of vocab
Found embeddings for  98.88% of all text
Number of unkown words in cbow =  948


[('ගොසිප්පෝ', 29),
 ('ගොසිප්පා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('නැකපතියා', 7),
 ('හදනගමං', 5),
 ('වලලං', 5),
 ('තෙවා', 4),
 ('නැකපති', 4),
 ('නැකපතියො', 4),
 ('ඩෙංගුදැයි', 4),
 ('ලොල්ලෙක්නෙ', 4),
 ('පික්ස්සු', 3),
 ('ලොල්ලනෙ', 3),
 ('නැගිටහල්ලා', 3),
 ('කුණුහර්පවලින්නෙ', 3),
 ('ප්\u200dරද්ශවාසී', 3),
 ('වහල්ලුනෙ', 3),
 ('ප්\u200dරොෙජට්', 3),
 ('ලෝගා', 3),
 ('හම්සව', 3)]

In [72]:
# word length 3
embed_cbow = get_emb_index(model_cbow)
embed_cbow_stem_3 = add_stem(embed_cbow,oov_cbow)
oov_cbow_3 = check_coverage(vocab_p, embed_cbow_stem_3)
print("Number of unkown words in cbow = ", len(oov_cbow_3))
oov_cbow_3[:20]

found 2137729 word vectors
Added 692 words to embedding with longer suffix
Added 39 words to embedding with shorter suffix
2138460
Found embeddings for 96.14% of vocab
Found embeddings for  98.99% of all text
Number of unkown words in cbow =  853


[('ගොසිප්පෝ', 29),
 ('ගොසිප්පා', 16),
 ('කෙනෙහිලිකන්', 14),
 ('නැකපතියා', 7),
 ('හදනගමං', 5),
 ('වලලං', 5),
 ('නැකපති', 4),
 ('නැකපතියො', 4),
 ('ඩෙංගුදැයි', 4),
 ('ලොල්ලෙක්නෙ', 4),
 ('පික්ස්සු', 3),
 ('ලොල්ලනෙ', 3),
 ('නැගිටහල්ලා', 3),
 ('කුණුහර්පවලින්නෙ', 3),
 ('ප්\u200dරද්ශවාසී', 3),
 ('වහල්ලුනෙ', 3),
 ('ප්\u200dරොෙජට්', 3),
 ('කිරීඩකයෝ', 2),
 ('අභයාරාමයෙදි', 2),
 ('ද්ව්යන්ගනවක්', 2)]

### Further evaluation on DL model

In [ ]:
def make_data(X):
    t = Tokenizer(num_words=len_voc)
    t.fit_on_texts(X)
    X = t.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=max_len)
    return X, t.word_index

In [ ]:
X, word_index = make_data(train['question_text'])

In [ ]:
def make_embed_matrix(embeddings_index, word_index, len_voc):
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = word_index
    embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [ ]:

embedding = make_embed_matrix(embed_glove, word_index, len_voc)

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def make_model(embedding_matrix, embed_size=300, loss='binary_crossentropy'):
    inp    = Input(shape=(max_len,))
    x      = Embedding(len_voc, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x      = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x      = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pl = GlobalAveragePooling1D()(x)
    max_pl = GlobalMaxPooling1D()(x)
    concat = concatenate([avg_pl, max_pl])
    dense  = Dense(64, activation="relu")(concat)
    drop   = Dropout(0.1)(concat)
    output = Dense(1, activation="sigmoid")(concat)
    
    model  = Model(inputs=inp, outputs=output)
    model.compile(loss=loss, optimizer=Adam(lr=0.0001), metrics=['accuracy', f1])
    return model

In [ ]:
checkpoints = ModelCheckpoint('weights.hdf5', monitor="val_f1", mode="max", verbose=True, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, verbose=1, min_lr=0.000001)

In [ ]:
pred_val = model.predict(X_val, batch_size=512, verbose=1)